In [1]:
import time
import json
from flask import Flask, request
app = Flask(__name__)
import re

In [ ]:
 # Opening JSON file
f = open(dir + '/data.json')
 
# returns JSON object as 
# a dictionary
all_tables_dict= json.load(f)
 
# Iterating through the json
# list

# Closing file
f.close()

In [367]:
tables_list = ["app_slp_customer", "app_visits", "app_browsing_anonymous_user_city", "app_browsing_user_city",
               "customer_details", "tataneu_parameters", "microsegments_vw_ihcl","currency_conversion_fact_vw","appjourneyandmicrosegment",
                 "microsegments_vw_bigbasket", "microsegments_vw_PvTML", "Transaction", "Items","Transaction_earn_burn",
                 "loyalty_total_earn_burn", "Available_Neucoins", "brand_transaction_organic_inorganic","brand_transaction_metrics",
                 "brand_bb_memberid_withoutmemberid","marketing_cohorts_cohort_tagging"]

In [340]:
table_dict = {}

for key in dict_final.keys:
    l = dict_final[key]
    for i in l:
        if i in table_dict[key].columns.to_list():
            table_dict[key][i][1] = True

In [330]:
tables_list2 = []
for table in tables_list:
    table2 = f"{table} = pd.read_csv('{table}.csv')"
    tables_list2.append(table2)

In [369]:
import pandas as pd
for table in tables_list2:
    exec(table)

In [368]:
tables_list_nonstring = []
all_attributes = {}
for table in tables_list:
    attr = eval(f"{table}.columns[1:].to_list()")
    all_attributes[table] = attr
    table2 = exec(table)
    tables_list_nonstring.append(table2)

In [326]:
context = "I want the cohort to target cross category or cross brand users."

In [353]:
campaginName = "Marketing Campaign"
campaignDetails = "I want the cohort to target cross category or cross brand users."
campaignSize = "500"
minNoOfCohort =  1
maxNoOfCohort = 5
minCohortSize = 10

In [362]:
@app.route('/api/model_input', methods=['POST','GET'])
def model_input():
    dict_input = {"campaignName": campaignName,
                    "campaignDetails": context,
                    "campaignSize":campaignSize,
                    "minNoOfCohort": minNoOfCohort,
                    "maxNoOfCohort":maxNoOfCohort,
                    "minCohortSize":minCohortSize,
                    }
    return dict_input

In [325]:
def gptcall(attributes, context, campaignName, minCohortSize,minNoOfCohort,maxNoOfCohort):
    payload = json.dumps({
        "messages": [
        {
            "role": "system",
            "content": f'''Given the dictionary all_attributes{attributes} with keys as table names and values as a list
    of columns or attributes corresponding to that table,given the context{context} which is the objective of the
    marketing campaign called as {campaignName} identify from the given SQL query the  
    names/attributes which are relevant to the given query besides the one mentioned in the query itself in order to create a 
    marketing segment or cohort of minimum size {minCohortSize}from which further analyses can
    be done. The minimum number of tables to be used is {minNoOfCohort} and maximum number of tables to be used is {maxNoOfCohort}.
    Give a reason for your choice of attributes and how these attributes would create a good marketing
    segment or cohort Give the output as a dictionary with key as the corresponding table from which it is taken 
    and values as a list of attributes in that table.Call the dictionary
    as dict_final. Enclose the dictionary in triple backquotes. Give the explanation seperately in numbered bullet
    points in a dictionary called dict_explanation with keys as attributes and columns as the explanation for the same.
    Dont use apostrophe for words in the explanation. Give a name for the marketing cohort as a dictionary
    dict_cohort with key as the name and value as explnation for why you have given that name.  Give the final
    result in python syntax.'''
        },
    #     {
    #       "role": "user",
    #       "content": "Give top 10 customer spend."
    #     },
    #     {
    #       "role": "assistant",
    #       "content": "'''sql SELECT customer_hash, FROM bb_sales, GROUP BY customer_hash, ORDER BY order_value DESC, LIMIT 10;'''"
    #     },
        {
            "role": "user",
            "content": f'''SELECT * FROM microsegments_vw_bigbasket  WHERE 	bigbasketMonetaryValue > 1000'''

        }],
        
        "max_tokens": 4096,
        "temperature": 0.0,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "top_p": 0.95,
        "stop": None
    })
    headers = {
        'Content-Type': 'application/json',
        'api-key': 'b34db39a21bd4d6aa73f5a12e9f6fdf9'
    }

    import requests
    time1 = time.time()
    url = '''https://tdl-aoai-southindia.openai.azure.com/openai/deployments/tdl-gpt4-turbo-preview/chat/completions?api-version=2023-12-01-preview'''
    response = requests.request("POST", url, headers=headers, data=payload)

    result = response.text
    time2 = time.time()
    total_time = time2 - time1
    return (result, total_time)

In [336]:
openai_result = gptcall(all_attributes, context)

In [ ]:
 Give the explanation seperately in numbered bullet
    points in a dictionary called dict_explanation with keys as attributes and columns as the explanation for the same.
    Dont use apostrophe for words in the explanation. Give a name for the marketing cohort as a dictionary
    dict_cohort with key as the name and value as explnation for why you have given that name.

In [337]:
def postprocessing(gpt_output):
    matches4 = [m.group(1) for m in re.finditer("```([\w\W]*?)```", gpt_output)]
    matches4[0] = matches4[0].replace("python", "")
    matches4[0] = matches4[0].replace("\\n", "\n")
    matches4[0] = matches4[0].replace('\\', "")
    return matches4[0]

In [338]:
#Execute python statement and create dictionary of tables and features!
exec(postprocessing(openai_result[0]))

In [370]:
table_dict = {}

for key in dict_final.keys():
    l = dict_final[key]
    for i in l:
        if i in all_tables_dict[key].keys():
            all_tables_dict[key][i][1] = True

In [371]:
all_features = []
for key in dict_final.keys():
    for feat in dict_final[key]:
        all_features.append(feat)

In [373]:
@app.route('/api/final_call_output', methods=['POST','GET'])
def final_call_output():
    dict_output = {"campaignName": campaignName,
                    "campaignDetails": campaignDetails,
                    "campaignSize":campaignSize,
                    "minNoOfCohort": minNoOfCohort,
                    "maxNoOfCohort":maxNoOfCohort,
                    "minCohortSize":minCohortSize,
                    "features" : all_features,
                    "feature_1":all_tables_dict
                    }
    return dict_output

In [321]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Feb/2024 14:47:36] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [23/Feb/2024 14:47:36] "GET /favicon.ico HTTP/1.1" 404 -
